In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
train = pd.read_csv('C:/222/large_data/kaggle/mnist/train.csv')
test = pd.read_csv('C:/222/large_data/kaggle/mnist/test.csv')
y = train['label']

In [3]:
train = train.drop(columns=['label'])

In [4]:
batch_size = 16
classes = y.unique()
num_classes = np.shape(classes)[0]

In [5]:
# def onedarr2imgarr(onedarr):
#     imgarr = np.zeros((np.shape(onedarr)[0], 28, 28))
#     for k in range(np.shape(onedarr)[0]):
#         if k%2000 == 0:
#             print(k, np.shape(onedarr)[0])
#         for i in range(28):
#             for j in range(28):
#                 imgarr[k, i, j] = onedarr.iloc[k][28 * i + j]
        
#     return imgarr

In [6]:
train_data = np.array(train).reshape(-1, 28, 28)
test_data = np.array(test).reshape(-1, 28, 28)

In [7]:
X_train = train_data.reshape(-1, 28, 28, 1)
X_test = test_data.reshape(-1, 28, 28, 1)

In [8]:
# plt.imshow(test_data[12], cmap = 'gray')

In [9]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))
 
def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

In [13]:
np.shape(X_train[0])

(28, 28, 1)

In [15]:
Image.imshow(np.shape(X_train[0]))

AttributeError: module 'PIL.Image' has no attribute 'imshow'

In [18]:
def create_convolutional_layer(input,
               num_input_channels, 
               conv_filter_size,        
               num_filters):  
    
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(num_filters)

    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')

    layer += biases

    ## We shall be using max-pooling.  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    ## Output of pooling is fed to Relu which is the activation function for us.
    layer = tf.nn.relu(layer)
    return layer

In [12]:
def create_flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])
    return layer

In [ ]:
layer_shape = layer.get_shape()
num_features = layer_shape[1:4].num_elements()

In [13]:
def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    """
    creating fully-connected layer 
    """
    
    #Let's define trainable weights and biases.
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)
 
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
 
    return layer

In [102]:
x = tf.placeholder(tf.float32, shape=[None, 28, 28 , 1], name='x')
y_true = tf.placeholder(tf.float32, shape=[None, len(y)], name='y_true')
# y_true_cls = tf.argmax(y_true, dimension=10)

In [103]:
layer_c1 = create_convolutional_layer(input = x,
               num_input_channels = 1, 
               conv_filter_size = (3),        
               num_filters = 1)
layer_fl = create_flatten_layer(layer_c1)
layer_out = create_fc_layer(layer_fl,          
             num_inputs = (layer_fl.get_shape()[1:].num_elements()),    
             num_outputs = num_classes,
             use_relu = True)

In [104]:
y_pred = tf.nn.softmax(layer_out, name = "y_pred")
# y_pred_cls = tf.argmax(y_pred, dimension = 10)

In [105]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_pred,
                                                    labels=y_true)
cost = tf.reduce_mean(cross_entropy)

In [106]:
sess = tf.Session()

In [107]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [108]:
# batch_size = 16

feed_dict_train = {x: X_train,
                           y_true: y}
 
sess.run(optimizer, feed_dict=feed_dict_train)

ValueError: Cannot feed value of shape (42000,) for Tensor 'y_true_5:0', which has shape '(?, 42000)'